In [1]:
import pandas as pd

# Práca
bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\df_bridges_2024_all.csv"

# Load the CSV data into a DataFrame
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')

In [2]:
# batch = filtered_bridges[filtered_bridges["Číslo_PK"] == "2753"]
# for _, item in batch.iterrows():
#     print(f'{item["ID_mosta"]};{item["Zemepisná_šírka"]}, {
#           item["Zemepisná_dĺžka"]}; {item["Dĺžka_premostenia"]}')

In [3]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')

In [4]:
# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
filtered_bridges = df_bridges_all[

    (df_bridges_all["Správca_úseku"] == 'BBRSC-STRED') |
    (df_bridges_all["Správca_úseku"] == 'BBRSC-SEVER') |
    (df_bridges_all["Správca_úseku"] == 'BBRSC-JUH')
]

# Reindex the DataFrame
filtered_bridges.reset_index(drop=True, inplace=True)

filtered_bridges = filtered_bridges[filtered_bridges["n_2023"] != 0]

filtered_bridges = filtered_bridges[filtered_bridges["Material"]
                                    == "prefabrikovaný predpätý betón"]

filtered_bridges = filtered_bridges[
    (filtered_bridges["n_2024"] == 4) |
    (filtered_bridges["n_2024"] == 5)
]

# Replace double space with single space in each item of the "Predmet_premostenia" column
filtered_bridges["Predmet_premostenia"] = filtered_bridges["Predmet_premostenia"].str.replace(
    "  ", " ")


# Display the filtered DataFrame
filtered_bridges.shape

(111, 44)

In [5]:
import pandas as pd

path = r"C:\Users\relia\Documents\GitHub\Bridges\06_SC_BB\IoT\Mosty.xlsx"

# Load the Excel file into a DataFrame
df = pd.read_excel(path)

# Display the first few rows to ensure it loaded correctly
print(df.head())

  ID_mosta                 GPS  Dlzka Intenzita   PK
0    M4403  48.07523, 19.17479  14.60      1472  527
1    M2559  48.07037, 19.25055  12.00      1383  527
2     M403  48.07406, 19.26022  15.30      1383  527
3    M6025  48.16523, 19.35814  17.00      3860  527
4     M381   48.1684, 19.36058  15.78      3860  527


In [6]:
df.columns

Index(['ID_mosta', 'GPS', 'Dlzka', 'Intenzita', 'PK'], dtype='object')

In [13]:
import os

folder_path = r"C:\Users\relia\Documents\GitHub\Bridges\06_SC_BB\IoT\Routes"

# List all files in the folder
files = os.listdir(folder_path)

# Define the bridge ID
bridge_id = "M7230"

# Filter the files for the specific bridge
clear_route = None
alternative_route = None

for file in files:
    if file.startswith(bridge_id):
        if "_clear" in file:
            clear_route = os.path.join(folder_path, file)
        elif "_alt" in file:
            alternative_route = os.path.join(folder_path, file)

# Assign variables for clear and alternative routes
print(f"Clear route for {bridge_id}: {clear_route}")
print(f"Alternative route for {bridge_id}: {alternative_route}")

Clear route for M7230: C:\Users\relia\Documents\GitHub\Bridges\06_SC_BB\IoT\Routes\M7230_clear.gpx
Alternative route for M7230: C:\Users\relia\Documents\GitHub\Bridges\06_SC_BB\IoT\Routes\M7230_alt.gpx


In [15]:
import xml.etree.ElementTree as ET
import folium



def extract_coordinates(gpx_file):
    tree = ET.parse(gpx_file)
    root = tree.getroot()
    namespace = {"default": "http://www.topografix.com/GPX/1/1"}
    coordinates = []
    for trkpt in root.findall(".//default:trkpt", namespace):
        lat = float(trkpt.attrib['lat'])
        lon = float(trkpt.attrib['lon'])
        coordinates.append((lat, lon))
    return coordinates


# Extract coordinates for clear and alternative routes
clear_coordinates = extract_coordinates(clear_route)
alt_coordinates = extract_coordinates(alternative_route)

# Create a Folium map centered on the first point of the clear route
route_map = folium.Map(location=clear_coordinates[0], zoom_start=14)

# Add the clear route to the map
folium.PolyLine(clear_coordinates, color="blue", weight=5,
                opacity=0.8, tooltip="Clear Route").add_to(route_map)

# Add the alternative route to the map
folium.PolyLine(alt_coordinates, color="red", weight=5,
                opacity=0.8, tooltip="Alternative Route").add_to(route_map)

route_map

In [19]:
# Path to the GPX file
gpx_file = r"C:\Users\relia\Documents\GitHub\Bridges\06_SC_BB\IoT\Routes\M7230_alt.gpx"

# Read the GPX file as text
with open(gpx_file, "r", encoding="utf-8") as file:
    gpx_content = file.read()

# Extract <rtept> sections
# Split by <rtept> and ignore the first part (metadata)
rtept_sections = gpx_content.split("<rtept")[1:]

# Initialize totals
total_distance = 0.0
total_time = 0.0

# Process each <rtept>
for rtept in rtept_sections:
    # Extract <gh:distance> value
    if "<gh:distance>" in rtept:
        start_distance = rtept.find("<gh:distance>") + len("<gh:distance>")
        end_distance = rtept.find("</gh:distance>")
        distance = float(rtept[start_distance:end_distance])
        total_distance += distance

    # Extract <gh:time> value
    if "<gh:time>" in rtept:
        start_time = rtept.find("<gh:time>") + len("<gh:time>")
        end_time = rtept.find("</gh:time>")
        time = float(rtept[start_time:end_time])
        total_time += time

# Convert totals to kilometers and minutes
total_distance_km = total_distance / 1000  # Convert meters to kilometers
# Convert milliseconds to minutes
total_time_minutes = total_time / (1000 * 60)


# Print results
print(f"Total Distance: {total_distance_km:.2f} kilometers")
print(f"Total Time: {total_time_minutes:.2f} minutes")

Total Distance: 13.87 kilometers
Total Time: 14.53 minutes


In [ ]:
import os

folder_path = r"C:\Users\relia\Documents\GitHub\Bridges\06_SC_BB\IoT\Routes"

# List all files in the folder
files = os.listdir(folder_path)

# Print the files
for file in files:
    print(file)

M1006_alt.gpx
M1006_clear.gpx
M1084_alt.gpx
M1084_clear.gpx
M1174_alt.gpx
M1174_clear.gpx
M123_alt.gpx
M123_clear.gpx
M1346_alt.gpx
M1346_clear.gpx
M2041_alt.gpx
M2041_clear.gpx
M2048_alt.gpx
M2048_clear.gpx
M2434_alt.gpx
M2434_clear.gpx
M2559_alt.gpx
M2559_clear.gpx
M2668_alt.gpx
M2668_clear.gpx
M2816_alt.gpx
M2816_clear.gpx
M2846_alt.gpx
M2846_clear.gpx
M2938_alt.gpx
M2938_clear.gpx
M2965_alt.gpx
M2965_clear.gpx
M3486_alt.gpx
M3486_clear.gpx
M3565_alt.gpx
M3565_clear.gpx
M369_alt.gpx
M369_clear.gpx
M381_alt.gpx
M381_clear.gpx
M3962_alt.gpx
M3962_clear.gpx
M403_alt.gpx
M403_clear.gpx
M4083_alt.gpx
M4083_clear.gpx
M4273_alt.gpx
M4273_clear.gpx
M4357_alt.gpx
M4357_clear.gpx
M4403_alt.gpx
M4403_clear.gpx
M4547_alt.gpx
M4547_clear.gpx
M454_alt.gpx
M454_clear.gpx
M465_alt.gpx
M465_clear.gpx
M5053_alt.gpx
M5053_clear.gpx
M5181_alt.gpx
M5181_clear.gpx
M5328_alt.gpx
M5328_clear.gpx
M5511_alt.gpx
M5511_clear.gpx
M5543_alt.gpx
M5543_clear.gpx
M5564_alt.gpx
M5564_clear.gpx
M5569_alt.gpx
M5569_cl